In [3]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
import scipy
from collections import OrderedDict
from scipy import interpolate
from skimage.draw import polygon
from abc import ABC, abstractmethod

from utils import *
import imageio

In [54]:
class Image:
    
    def __init__(self, name, imArr, pts = None):
        self.name = name
        self.imArr = imArr
        self.pts = pts
        self.height, self.width = imArr.shape[:2]

    def fromPath(pathName):
        imArr, imName = readImageNName(pathName)
        result = Image(imName, imArr)
        return result

    def setPts(self, pts, ptsSetup, withCorners = True):
        if withCorners:
            pts = np.vstack((pts, Image.getCorners(self.imArr)))
        self.pts = pts
        self.ptsSetup = ptsSetup
        return self.pts
    
    def setInputPts(self, ptSetup, withCorners = True):
        ptsCt = sum([x for x in ptSetup.values()])
        pts = np.array(getPoints(self, ptsCt))
        self.setPts(pts, ptSetup, withCorners)
        return self.pts
    
    def getCorners(imArr):
        print("Getting corners.")
        return np.array([(0, 0), (imArr.shape[1], 0), (0, imArr.shape[0]), (imArr.shape[1], imArr.shape[0])])
    
#     def refreshDims(self):
#         self.height, self.width = imArr.shape[:2]
    

In [7]:
imMan1, _ = readImageNName("sample_imgs/01-2m.jpg")

imM1 = Image.fromPath("sample_imgs/01-2m.jpg")
imM2 = Image.fromPath("sample_imgs/05-2m.jpg")

imM1PresetPts = np.array([(310.4090909090909, 415.6038961038961), (268.8506493506494, 397.42207792207785), (244.17532467532467, 361.0584415584415), (227.2922077922078, 314.3051948051948), (224.69480519480518, 236.38311688311683), (233.7857142857143, 185.73376623376618), (263.6558441558442, 148.0714285714285), (348.07142857142856, 144.1753246753246), (381.8376623376623, 177.94155844155836), (400.01948051948057, 235.08441558441552), (397.42207792207796, 309.1103896103896), (381.8376623376623, 351.9675324675324), (354.56493506493507, 394.8246753246753), (215.60389610389612, 310.4090909090909), (202.6168831168831, 270.1493506493506), (203.9155844155844, 235.08441558441552), (200.01948051948048, 181.83766233766227), (215.60389610389612, 133.78571428571422), (248.07142857142858, 98.72077922077915), (368.8506493506494, 98.72077922077915), (406.51298701298697, 135.08441558441552), (423.3961038961039, 179.24025974025966), (415.6038961038961, 238.98051948051943), (422.0974025974026, 279.24025974025966), (411.7077922077922, 315.6038961038961), (270.1493506493506, 242.8766233766233), (350.66883116883116, 244.1753246753246), (237.68181818181816, 228.590909090909), (254.56493506493504, 216.90259740259734), (298.72077922077926, 224.6948051948051), (327.2922077922078, 225.9935064935064), (362.3571428571429, 218.20129870129864), (387.03246753246754, 229.8896103896103), (310.4090909090909, 240.27922077922074), (283.1363636363636, 296.12337662337654), (311.7077922077922, 302.61688311688306), (335.08441558441564, 297.42207792207785), (257.1623376623377, 320.79870129870125), (310.4090909090909, 329.8896103896103), (357.1623376623377, 325.9935064935064), (274.0454545454545, 337.68181818181813), (309.1103896103896, 354.564935064935), (335.08441558441564, 344.17532467532465)])
imM2PresetPts = np.array([(327.2922077922078, 389.6298701298701), (279.2402597402597, 366.2532467532467), (250.6688311688312, 320.79870129870125), (242.87662337662337, 279.24025974025966), (246.77272727272728, 220.79870129870125), (255.86363636363635, 171.4480519480519), (289.62987012987014, 124.6948051948051), (390.92857142857144, 135.08441558441552), (415.6038961038961, 183.13636363636357), (424.6948051948052, 231.18831168831161), (419.5, 297.42207792207785), (406.51298701298697, 337.68181818181813), (379.2402597402597, 374.0454545454545), (236.38311688311686, 274.0454545454545), (235.08441558441555, 245.4740259740259), (236.38311688311686, 211.70779220779212), (242.87662337662337, 154.56493506493496), (259.7597402597403, 98.72077922077915), (310.4090909090909, 62.357142857142776), (377.9415584415584, 67.55194805194793), (419.5, 105.21428571428561), (433.78571428571433, 148.0714285714285), (431.1883116883117, 231.18831168831161), (428.5909090909091, 259.75974025974017), (424.6948051948052, 285.7337662337662), (297.42207792207796, 228.590909090909), (379.2402597402597, 235.08441558441552), (267.5519480519481, 215.60389610389603), (296.12337662337666, 206.51298701298697), (325.9935064935065, 211.70779220779212), (351.96753246753246, 218.20129870129864), (376.6428571428571, 211.70779220779212), (405.2142857142858, 220.79870129870125), (336.38311688311694, 228.590909090909), (309.1103896103896, 277.94155844155836), (338.98051948051955, 285.7337662337662), (362.3571428571429, 275.3441558441558), (287.03246753246754, 301.31818181818176), (336.38311688311694, 301.31818181818176), (371.448051948052, 305.21428571428567), (305.21428571428567, 316.90259740259734), (329.8896103896104, 323.39610389610385), (358.461038961039, 319.49999999999994)])

print(imM1PresetPts.shape)
print(getCorners(imM1.imArr).shape)

# imM1.pts = np.vstack((imM1PresetPts, getCorners(imM1.imArr)))
# imM2.pts = np.vstack((imM2PresetPts, getCorners(imM2.imArr)))


#### Defaults
mainPtSetup = OrderedDict()
mainPtSetup['wo'] = 13
mainPtSetup['bo'] = 12
mainPtSetup['go'] = 2
mainPtSetup['co'] = 6
mainPtSetup['ro'] = 4
mainPtSetup['yo'] = 6
# side pixels setup

(43, 2)


NameError: name 'getCorners' is not defined

In [8]:
imMyFace_neutral = Image.fromPath("../Photos/Regina_Ongowarsito_25605679_1.jpg")
imMyFace_smile = Image.fromPath("../Photos/Regina_Ongowarsito_25605679_2.jpg")

imMyFace_neutral.pts = np.array([(309.3766233766233, 678.1753246753246), (201.19480519480516, 656.0974025974026), (134.96103896103892, 594.2792207792207), (115.09090909090907, 510.3831168831168), (108.46753246753241, 344.7987012987013), (137.1688311688311, 232.2012987012987), (238.7272727272727, 150.51298701298697), (417.5584415584414, 163.75974025974017), (490.4155844155843, 238.82467532467524), (527.9480519480519, 355.8376623376623), (527.9480519480519, 514.7987012987012), (503.6623376623375, 600.9025974025974), (430.80519480519473, 660.512987012987), (97.42857142857139, 505.96753246753246), (77.55844155844147, 444.1493506493506), (75.35064935064929, 366.87662337662334), (86.38961038961031, 236.61688311688306), (163.66233766233762, 112.98051948051943), (247.55844155844142, 68.82467532467524), (410.9350649350649, 79.86363636363637), (497.03896103896085, 139.47402597402595), (563.272727272727, 247.65584415584408), (576.5194805194803, 371.2922077922077), (569.8961038961038, 444.1493506493506), (552.2337662337661, 503.7597402597402), (229.89610389610385, 358.0454545454545), (408.7272727272726, 355.8376623376623), (152.6233766233766, 311.68181818181813), (187.94805194805184, 278.56493506493507), (251.9740259740259, 276.3571428571428), (371.19480519480504, 282.98051948051943), (433.0129870129869, 280.77272727272725), (481.58441558441547, 311.68181818181813), (307.1688311688311, 353.62987012987014), (260.80519480519473, 468.43506493506493), (309.3766233766233, 479.47402597402595), (368.98701298701286, 472.8506493506493), (243.14285714285705, 558.9545454545455), (307.1688311688311, 510.3831168831168), (377.8181818181817, 543.5), (278.4675324675324, 558.9545454545455), (311.58441558441547, 567.7857142857142), (344.70129870129864, 567.7857142857142)])
imMyFace_smile.pts = np.array([(292.3939393939393, 677.0151515151515), (169.36363636363632, 647.3181818181818), (114.21212121212113, 577.3181818181818), (101.48484848484838, 488.2272727272726), (95.12121212121207, 337.6212121212119), (129.06060606060595, 233.6818181818179), (224.51515151515144, 148.83333333333303), (389.9696969696968, 157.31818181818153), (472.6969696969696, 240.04545454545428), (513.0, 352.46969696969677), (508.7575757575757, 496.7121212121211), (485.4242424242423, 581.560606060606), (417.54545454545445, 653.6818181818181), (73.90909090909082, 486.10606060606045), (65.42424242424232, 418.2272727272726), (69.66666666666657, 354.5909090909089), (82.39393939393932, 220.95454545454515), (156.63636363636357, 112.77272727272702), (216.03030303030295, 72.46969696969666), (415.4242424242423, 83.07575757575728), (459.9696969696968, 121.25757575757552), (527.8484848484848, 229.43939393939365), (536.3333333333333, 365.1969696969695), (534.212121212121, 428.83333333333314), (529.9696969696968, 490.3484848484847), (205.42424242424232, 352.46969696969677), (389.9696969696968, 352.46969696969677), (137.54545454545445, 312.1666666666664), (171.48484848484844, 286.712121212121), (252.09090909090895, 280.34848484848465), (360.2727272727272, 282.46969696969677), (434.51515151515144, 288.83333333333314), (468.4545454545453, 316.40909090909065), (296.63636363636357, 346.1060606060604), (241.48484848484844, 464.89393939393926), (294.51515151515144, 473.3787878787877), (351.7878787878787, 467.01515151515133), (213.90909090909082, 530.651515151515), (296.63636363636357, 513.681818181818), (372.99999999999994, 532.7727272727271), (249.96969696969694, 547.621212121212), (298.7575757575757, 558.2272727272725), (341.1818181818181, 556.1060606060605)])

imMyFace_neutral.pts = np.vstack((imMyFace_neutral.pts, Image.getCorners(imMyFace_neutral.imArr)))
imMyFace_smile.pts = np.vstack((imMyFace_smile.pts, Image.getCorners(imMyFace_smile.imArr)))

# neutralPts = imMyFace_neutral.setPts(mainPtSetup)
# smilePts = imMyFace_smile.setPts(mainPtSetup)

Getting corners.
Getting corners.


In [ ]:
dispPtsOnImg(imMyFace_neutral.imArr, imMyFace_neutral.pts)

In [9]:
def getPoints(im, n):
    n = int(n)
    plt.imshow(im.imArr)
    points = plt.ginput(n, timeout = 300, show_clicks=True)
    plt.close()
    print("points gotten from", im.name, ":", str(points))
    return np.array(points)

def dispPtsOnImg(imMan1, results):
    plt.imshow(imMan1)
    plt.plot(results[:, 0], results[:, 1], 'bo')
    for ind, tup in enumerate(results):
        x, y = tup
        plt.text(x, y, str(ind))
#     for x, y in results:
#         plt.text(x, y, "%d-%d" % (x,y))
    plt.waitforbuttonpress()
    plt.close()

In [10]:
def dispTriangulation(test_tri, im = None):
    if type(im) == np.ndarray:
        plt.imshow(im)
    plt.triplot(test_tri.points[:, 0], test_tri.points[:, 1], test_tri.simplices)
    # plt.plot(test_tri.points[:, 0], test_tri.points[:, 1])
    plt.waitforbuttonpress()
    plt.close()

In [11]:
def delaunaySmartCp(points, baseDela):
    assert len(points) == len(baseDela.points), "Number of points is not equal"
    # uses input pts, simplices
    
#     print("points:", points)
#     print("baseDela.points:", baseDela.points)
    
    newDela = Delaunay(baseDela.points.copy())
    newDela.simplices[:] = baseDela.simplices[:]
    newDela.points[:] = points
    
    return newDela
    

#def fnXX(im1, im2, ptSetup = mainPtSetup):
#     im1.setPts(ptSetup)
#     im2.setPts(ptSetup)

# fnXX(imM1, imM2)

In [ ]:
"""
given point P
find end_triangle of P
find corresponding start_triangle #should be the same index as end_triangle

calculate the transformation T from end triangle to start triangle

apply T to P to get P', which is P equivalent in start_triangle

P' is absolutely going to be in the middle of pixels, so interpolate it

put P' to P

"""






In [ ]:
dispPtsOnImg(im1.imArr, im1.pts)
dispPtsOnImg(im2.imArr, im2.pts)
im2Dela.find_simplex(np.array([289, 98])) # NOTE: x,y in matplotlib

In [12]:
plt.close()
# t1 = A
# t2 = X
clust1 = np.array([[1, 1], [4, 4], [6,1]])
# clust2 = np.array([[0, 1], [2, 1], [3,3]])
clust2 = np.array([[1, 1], [6, 6], [6,1]])
simp = np.array([[0, 1, 2]])
# clust2 = np.array([1, 2, 3])
plt.triplot(clust1[:, 0], clust1[:, 1], simp)
plt.triplot(clust2[:, 0], clust2[:, 1], simp)

def transformationMatrixFind(a_vects, x_vects):
    assert a_vects.shape == (3, 2) and x_vects.shape == (3, 2), "inputs are not triangle points of shape (3, 2)"
    
    matrixer = lambda p0, p1, p2: np.array([p0, p1, p2]).T
    # RETURNS TAKES X, Y FORMAT, RETURNS X, Y FORMAT [X, Y, 1]T

    A = np.vstack((a_vects.T, np.ones((1, 3))))
    X = np.vstack((x_vects.T, np.ones((1, 3))))
    
    try:
        Ainv = np.linalg.inv(A)
    except LinAlgError as e:
        print(A)
        raise e
    T = np.dot(X, Ainv)

    return T


### TEST
# triA = Delaunay(clust1)
startP = np.array((2, 1.75))
plt.plot(startP[0], startP[1], 'bo')

# scipy.spatial.tsearch(triA, startP)
stackV = lambda vect, i: np.hstack((vect, np.array([i])))
Ttt = transformationMatrixFind(clust1, clust2)

applyP = np.dot(Ttt, stackV(startP, 1))
plt.plot(applyP[0], applyP[1], 'go')

#, clust1, clust2

In [13]:
def warp2ArrResult(targetImage, sourceImage, triangulation):
    assert targetImage.imArr.shape[:2] == sourceImage.imArr.shape[:2]
    assert len(targetImage.pts) == len(sourceImage.pts) == len(triangulation.points), "correspondence points not equal between two input images"
    
    finalTargetImageArr = targetImage.imArr.copy()
    
    # A = our target stuff
    # X = our source stuff
    
    source_Rchan = scipy.interpolate.interp2d(range(sourceImage.width), range(sourceImage.height), sourceImage.imArr[:,:,0])
    source_Gchan = scipy.interpolate.interp2d(range(sourceImage.width), range(sourceImage.height), sourceImage.imArr[:,:,1])
    source_Bchan = scipy.interpolate.interp2d(range(sourceImage.width), range(sourceImage.height), sourceImage.imArr[:,:,2])

    triangles = triangulation.simplices
    matrices = []
    
    yVals = []
    xVals = []
    channel = []
    vals = []
#     rChanVals = []
#     gChanVals = []
#     bChanVals = []

    for i in range(len(triangles)): # range(len(triangles)):
        tri_indexes = triangles[i]
        targetTriangle = targetImage.pts[tri_indexes]
        sourceTriangle = sourceImage.pts[tri_indexes]

        transf = transformationMatrixFind(targetTriangle, sourceTriangle)


        polyRes = polygon(targetTriangle[:, 0], targetTriangle[:, 1])
        Aline = np.vstack((polyRes[0], polyRes[1]))

        Xline = np.dot(transf, np.vstack((Aline, np.ones(Aline.shape[1]))))[:2]


        for i in range(Aline.shape[1]):
            currA = Aline[:, i]
            currX = Xline[:, i]
            # print(currA, currX)
            
            yVals[]
            
            finalTargetImageArr[int(currA[1]), int(currA[0]),:] = [source_Rchan(currX[0], currX[1]), source_Gchan(currX[0], currX[1]), source_Bchan(currX[0], currX[1])]
            
    return finalTargetImageArr


def morph(im1, im2, warpAlpha, dissolveAlpha, midPtsDela):
    assert len(im1.pts) == len(im2.pts)
    
    warpAlpha1 = warpAlpha
    warpAlpha2 = 1 - warpAlpha
    midPts = np.dot(im1.pts, warpAlpha1) + np.dot(im2.pts, warpAlpha2)
    midWay = Image("mid_img", np.zeros(list(im1.imArr.shape[:2]) + [3]), midPts)

    midWay1Arr = warp2ArrResult(midWay, im1, midPtsDela)
    midWay2Arr = warp2ArrResult(midWay, im2, midPtsDela)

    disAlpha1 = dissolveAlpha
    disAlpha2 = 1 - dissolveAlpha
    assert disAlpha1 + disAlpha2 == 1.0

    midWay.imArr = midWay1Arr * disAlpha1 + midWay2Arr * disAlpha2
    
    return midWay


def createMidwayDela(im1, im2):
    assert len(im1.pts) == len(im2.pts)
    return Delaunay(np.dot(im1.pts + im2.pts, 0.5))

# midPtsDela = createMidwayDela(imM1, imM2)
# plt.imshow(morph(imM1, imM2, 0.5, 0.5, midPtsDela).imArr)


# midWay.imArr
    

In [61]:
class FaceDb(ABC):
    
    allDatabases = dict()
    
    @property
    @abstractmethod
    def ptsSetup(self):
        pass
    
    
    def __init__(self, dbName, directoryPath, imgformat, ptsformat):#, fileReader):
        FaceDb.allDatabases[dbName] = self
        self.name = dbName
        # self.ptsSetup = ptsSetup
        self.directoryPath = directoryPath
#         self.fileReader = fileReader
        self.imgformat = imgformat
        self.ptsformat = ptsformat
        self.allIms = dict()
    
    def createImg(self, filename):
        im = Image.fromPath("%s/%s" % (self.directoryPath, filename))
        self.allIms[im.name] = im
        return im
    
    @abstractmethod
    def readPts(self, filename):
        pass
    
    def createImgWCorrPts(self, imgName, withCorners = True):
        im = self.createImg("%s.%s" % (imgName, self.imgformat))
        ptsRes = self.readPts("%s.%s" % (imgName, self.ptsformat))
        im.setPts(ptsRes, self.ptsSetup, withCorners)
        return im
        
    def getImgNames(self):
        rawFileList = os.listdir(self.directoryPath)
        namesFileList = []
        for f in rawFileList:
            fullName = os.path.splitext(f)
            if fullName[1] == ".%s" % self.imgformat:
                namesFileList.append(fullName[0])
        return set(namesFileList)

    def getAllImsWPts(self):
        fileNames = self.getImgNames()

        for fileName in fileNames:
            im = self.createImgWCorrPts(fileName)
        return self.allIms


    

In [62]:
class DanesDb(FaceDb):
    
    ptsSetup = OrderedDict()
    ptsSetup["jaw"] = 13
    ptsSetup["right_eye"] = 8
    ptsSetup["left_eye"] = 8
    ptsSetup["right_eyebrow"] = 5
    ptsSetup["left_eyebrow"] = 5
    ptsSetup["mouth"] = 8
    ptsSetup["nose"] = 11
    
    def readPts(self, filename):
        # def danesPtsReader(fileName):
        # fileName = "../imm_face_db/01-1m.asf"
        fullPath = "%s/%s" % (self.directoryPath, filename)
        rawPtsInput = fileToString(fullPath).split("\n")[16:-6]
        listFormatInput = [line.split(" \t") for line in rawPtsInput]
        xYcoordTupsRelative = [(float(line[2]), float(line[3])) for line in listFormatInput]
        xYcoordTupsFinal = [(x * 640, y * 480) for x, y in xYcoordTupsRelative]
        return np.array(xYcoordTupsFinal)

In [63]:
testDanesDb = DanesDb("test", "../imm_face_db", "jpg", "asf")

In [64]:
testDanesDb.getAllImsWPts()

Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corners.
Getting corner

{'01-1m': <__main__.Image at 0xda3e69588>,
 '01-2m': <__main__.Image at 0xd9c934a90>,
 '01-3m': <__main__.Image at 0xda3e69ef0>,
 '01-4m': <__main__.Image at 0xda296bda0>,
 '01-5m': <__main__.Image at 0xda296bf60>,
 '01-6m': <__main__.Image at 0xda2952dd8>,
 '02-1m': <__main__.Image at 0xda298ec88>,
 '02-2m': <__main__.Image at 0xda298e898>,
 '02-3m': <__main__.Image at 0xda11c77b8>,
 '02-4m': <__main__.Image at 0xdb0106f28>,
 '02-5m': <__main__.Image at 0xda2952780>,
 '02-6m': <__main__.Image at 0xda298e198>,
 '03-1m': <__main__.Image at 0xda298e320>,
 '03-2m': <__main__.Image at 0xdb00fa278>,
 '03-3m': <__main__.Image at 0xda298e518>,
 '03-4m': <__main__.Image at 0xda29527b8>,
 '03-5m': <__main__.Image at 0xda3e69470>,
 '03-6m': <__main__.Image at 0xda2952f98>,
 '04-1m': <__main__.Image at 0xdb011f208>,
 '04-2m': <__main__.Image at 0xd963c9e10>,
 '04-3m': <__main__.Image at 0xda296bcc0>,
 '04-4m': <__main__.Image at 0xdaaf0a278>,
 '04-5m': <__main__.Image at 0xdb0106c88>,
 '04-6m': <

In [50]:
testDanesDb.readPts("08-1f.asf")

array([[225.6612992, 290.8057584],
       [229.5322624, 310.7990448],
       [239.2096704, 341.7563712],
       [254.048384 , 370.1339136],
       [277.2741888, 386.9024544],
       [293.4032256, 395.2867392],
       [313.4032256, 400.4462832],
       [344.3709568, 394.6417824],
       [360.4999936, 384.967632 ],
       [382.4354944, 369.4889568],
       [394.048384 , 350.1406272],
       [402.4354944, 321.1181568],
       [410.1774208, 291.45072  ],
       [384.3709568, 239.2102128],
       [374.693568 , 235.3405536],
       [366.9515968, 232.1158368],
       [355.9838848, 234.0506592],
       [345.0161344, 243.0798912],
       [354.693568 , 248.2394304],
       [367.5967808, 250.8192144],
       [376.6290304, 247.5944976],
       [254.6935488, 237.9203376],
       [265.6612992, 234.0506592],
       [274.6935488, 232.760784 ],
       [285.6612992, 237.9203376],
       [294.6935488, 243.724824 ],
       [285.6612992, 247.5944976],
       [274.6935488, 247.5944976],
       [263.7257984,

In [65]:
check = testDanesDb.allIms["08-1f"]
print(check.pts)
dispPtsOnImgWSetup(check.imArr, check.pts, DanesDb.ptsSetup)

[[225.6612992 290.8057584]
 [229.5322624 310.7990448]
 [239.2096704 341.7563712]
 [254.048384  370.1339136]
 [277.2741888 386.9024544]
 [293.4032256 395.2867392]
 [313.4032256 400.4462832]
 [344.3709568 394.6417824]
 [360.4999936 384.967632 ]
 [382.4354944 369.4889568]
 [394.048384  350.1406272]
 [402.4354944 321.1181568]
 [410.1774208 291.45072  ]
 [384.3709568 239.2102128]
 [374.693568  235.3405536]
 [366.9515968 232.1158368]
 [355.9838848 234.0506592]
 [345.0161344 243.0798912]
 [354.693568  248.2394304]
 [367.5967808 250.8192144]
 [376.6290304 247.5944976]
 [254.6935488 237.9203376]
 [265.6612992 234.0506592]
 [274.6935488 232.760784 ]
 [285.6612992 237.9203376]
 [294.6935488 243.724824 ]
 [285.6612992 247.5944976]
 [274.6935488 247.5944976]
 [263.7257984 245.0147136]
 [346.3064576 219.861888 ]
 [361.1451712 212.7674976]
 [375.3387072 208.8978336]
 [385.0161344 210.8326704]
 [399.854848  216.6371712]
 [297.9193472 217.9270656]
 [283.0806528 210.187728 ]
 [266.3064576 204.3832272]
 

In [66]:
allIms = list(testDanesDb.allIms.values())
imWidth = 640
imHeight = 480

finalIm = Image("final_im", np.zeros((480, 640, 3)))

for i in range(len(allIms)):
    im = allIms[i]
#     print(im.name)
#     dispPtsOnImgWSetup(im.imArr, im.pts, DanesDb.ptsSetup)
#     plt.waitforbuttonpress()
#     plt.close()
    if i == 0:
        finalIm.pts = im.pts.copy()
    else:
        finalIm.pts += im.pts
        
finalIm.pts = finalIm.pts / len(allIms)

In [67]:
dispPtsOnImgWSetup(finalIm.imArr, finalIm.pts, DanesDb.ptsSetup)

In [ ]:
midDela = Delaunay(finalIm.pts)
for im in allIms:
    # midDela = createMidwayDela(im1, im2)
    newImArr = warp2ArrResult(finalIm, im, midDela)
    finalIm.imArr += newImArr

finalIm.imArr /= len(allIms)

In [29]:
plt.imshow(finalIm.imArr)

# Danes Reader

In [18]:
def fileToString(fileName):
    f = open(fileName, "r+")
    content = f.read()
    f.close()
    return content

# print(sum([x for x in danesDbPtsSetup.values()]))
# danesDb = FaceDb("Danes", )

def dispPtsOnImgWSetup(im, results, ptsSetup):
    assert type(ptsSetup) == OrderedDict
    plt.close()
    plt.imshow(im)
    
    start = 0
    for label, ct in ptsSetup.items():
        end = start + ct
        plt.plot(results[start:end, 0], results[start:end, 1], 'o')
        start = end
#     for x, y in results:
#         plt.text(x, y, "%d-%d" % (x,y))
    for ind, coord in enumerate(results):
        x, y = coord
        plt.text(x, y, str(ind))

In [ ]:
# np.array(file011mPts)
dispPtsOnImgWSetup(imM1.imArr, np.array(file011mPts), danesDbPtsSetup)

# Morph Sequence

In [ ]:
### createMorph

def createMorphSequence(im1, im2, steps):
    """
    create 
    """
    # assert 0 <= step and step <= 1, "step value is invalid: %s" % step
    assert 0 < steps, "steps value is invalid: %s" % step 
    assert len(im1.pts) == len(im2.pts)

    midDela = createMidwayDela(im1, im2)
    
    finalStack = []
    
    for alpha in np.linspace(0, 1.0, steps):
        print("Start morphing alpha: ", alpha)
        result = morph(im1, im2, alpha, alpha, midDela)
        print("End morphing alpha: ", alpha)
        finalStack.append(result)
        
    return finalStack
    

In [ ]:
resss = createMorphSequence(imM1, imM2, 6)

In [ ]:
for im in resss:
    plt.imshow(im.imArr)
    plt.waitforbuttonpress()
    plt.close()

In [ ]:
def convertToGif(fileName, lstImArr):
    imageio.mimsave('./%s/%s.gif' % (fOutputDirectory, fileName), [im.imArr for im in lstImArr])
    return
convertToGif("test", resss)

In [ ]:
im1 = imM1
im2 = imM2
warpAlpha = 0.5
dissolveAlpha = 0.5

# dispPtsOnImg(im2.imArr, im2.pts)
# dispPtsOnImg(im1.imArr, im1.pts)

assert im1.imArr.shape[:2] == sourceImage.imArr.shape[:2]

# im1Dela = delaunaySmartCp(im1.pts, midPtsDela)
# print(im1Dela.points, im1.pts)
# im1.pts[0] = [1, 1]
# print(im1Dela.points, im1.pts)
# im2Dela = delaunaySmartCp(im2.pts, midPtsDela)

dispTriangulation(im1Dela,  im1.imArr)
dispTriangulation(im2Dela, im2.imArr)
dispTriangulation(midPtsDela, im1.imArr)
dispTriangulation(midPtsDela, im2.imArr)

In [ ]:
test_tri = Delaunay(results)
x_results = [x[0] for x in results]
y_results = [x[1] for x in results]
plt.triplot(x_results, y_results, test_tri.simplices)
plt.plot(x_results, y_results, 'bo')

test_tri.points[:]= np.dot(test_tri.points, np.roll(np.eye(2), 1, axis=1))
plt.triplot(test_tri.points[:, 0], test_tri.points[:, 1], test_tri.simplices)
plt.plot(test_tri.points[:, 0], test_tri.points[:, 1], 'ro')

In [ ]:
plt.close()
import random
x = [0,1,2]
y = [0,3]
z = [[1,2,3], [4,5,6]]
f = interpolate.interp2d(x, y, z, kind='linear')
import matplotlib.pyplot as plt
# xnew = np.arange(-5.01, 5.01, 1e-2)
# ynew = np.arange(-5.01, 5.01, 1e-2)
znew = f([1.4], [2])
znew
#znew = f(xnew, ynew)
# plt.plot(xnew, znew[0, :], 'bo-')
# plt.show()
# znew

In [ ]:
img = np.zeros((10, 10), dtype=np.uint8)
r = np.array([1, 2, 8, 1])
c = np.array([1, 7, 4, 1])
rr, cc = polygon(r, c)
img[rr, cc] = 1
img, rr, cc

In [ ]:
# r_only = im1.imArr[:,:,0]
# print("r_only:\n", r_only)
# r_flat = r_only.T.reshape(im1.width * im1.height)
r_channel = scipy.interpolate.interp2d(range(im1.width), range(im1.height), im1.imArr[:,:,0])
print(r_chan(0, 1))
print(im1.imArr.shape)

# print(r_flat[im1.width])
meow_start = 0
iSte = 0
# while iSte != im1.height:
#     meow_end = iSte + 1 * im1.width
#     print(r_flat[meow_start:meow_end])
#     print()
#     print(im1.imArr[iSte, :, 0])
#     print("---")
#     meow_start = meow_end
#     iSte +=1
print(im1.imArr[0, 0, 0], im1.imArr[0, 1, 0], im1.imArr[1, 0, 0], im1.imArr[1, 1, 0])

In [ ]:
testtt = np.array([
    [[1, 2, 3], [4, 5, 6]],
    [[7, 8, 9], [10, 11, 12]]
])

testtt.reshape(testtt.shape[0] * testtt.shape[1], 3)[:, 0]

In [ ]:
(0.10196078 + 0.0745098 + 0.08627451 +  0.07843137) / 4

In [ ]:
imInterp = im1
px, py = (0, 0)

red = im1.imArr[:, :, 0]
# blue = im1[:, :, 1]
# green = im1[:, :, 1]
# redPoints = [red[py, px], red[py, px+1], red[py+1, px], red[py+1, px+1]]
redPoints = [0, 1, 2, 3]
r = scipy.interpolate.interp2d([px, px+1], [py, py+1], redPoints)
r(1, 0)